1. EDA

2. Define utils of dataset

In [9]:
def pair_comparision(a,b):
    if a == b:
        return 1
    else:
        return 0

In [10]:
def gram_matrix(list_of_score):
    n = len(list_of_score)
    gram_matrix = [[0 for _ in range(n)] for _ in range(n)]

    for i in range(n):
        for j in range(n):
            gram_matrix[i][j] = pair_comparision(list_of_score[i], list_of_score[j])
    
    return gram_matrix

In [11]:
# Test
list_of_score = [1,2,3,2,3,4,2,1,4]
gram_matrix(list_of_score)

[[1, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 1, 0, 1, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 1, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 1],
 [0, 1, 0, 1, 0, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 1]]

In [12]:
# Define dataset

2. Model

In [13]:
from torch import nn
import torch
import torchvision.models as models
import timm

class Setting_2_model(nn.Module):
    def __init__(self, model_name: str, embed_dim: int):
        super(Setting_2_model, self).__init__()

        # Load the specified pre-trained model
        if model_name.startswith('resnet'):
            if model_name == 'resnet50':
                self.model = models.resnet50(pretrained=True)
            elif model_name == 'resnet101':
                self.model = models.resnet101(pretrained=True)
            elif model_name == 'resnet152':
                self.model = models.resnet152(pretrained=True)
            else:
                raise ValueError(f"Unsupported ResNet model: {model_name}")
                
            num_features = self.model.fc.in_features
            self.model.fc = nn.Linear(num_features, embed_dim)
        
        elif model_name.startswith('densenet'):
            if model_name == 'densenet121':
                self.model = models.densenet121(pretrained=True)
            else:
                raise ValueError(f"Unsupported DenseNet model: {model_name}")
                
            num_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(num_features, embed_dim)
        
        elif model_name.startswith('vit'):
            self.model = timm.create_model(model_name, pretrained=True)

            num_features = self.model.head.in_features
            self.model.head = nn.Linear(num_features, embed_dim)
        
        else:
            raise ValueError(f"Unsupported model: {model_name}")
    
    def forward(self, images):
        embeddings = []
        for image in images:
            image_embedding = self.model(image)
            embeddings.append(image_embedding)
        embeddings_tensor = torch.cat(embeddings, dim=0)
        embeddings_normalized = torch.nn.functional.normalize(embeddings_tensor, p=2, dim=1)

        # Tính gram matrix
        gram_matrix = torch.matmul(embeddings_normalized, embeddings_normalized.transpose(0, 1))
        return gram_matrix
    
    

3. Loss

In [14]:
from torch import nn

# Hàm tính Constrastive Learning Loss
class ConstrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ConstrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, gram_matrix_predicted, gram_matrix_ground_truth):
        # Tính khoảng cách Frobenius giữa hai ma trận Gram
        distance = torch.norm(gram_matrix_predicted - gram_matrix_ground_truth, p='fro')
        
        # Tính Constrastive Learning Loss
        loss = torch.clamp(distance - self.margin, min=0.0)
        return loss
 

In [17]:
# test
import torch

gt = [1,2,3,2,3,4,2,1,4]
gt = torch.tensor(gram_matrix(gt),dtype=torch.float)

pred = [1,2,3,2,3,4,2,2,4]
pred = torch.tensor(gram_matrix(pred), dtype=torch.float)

criterion = ConstrastiveLoss()
loss = criterion(pred, gt)

print(loss)

tensor(1.8284)
